# Load the pymoo result object saved with pickle.
ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

## Load result object
Place a pymoo result object file saved with pickle and execute the following cell.
Replace 'res_Cogent.pkl' with your result file saved with pickle.

In [2]:
with open('recent_files/res_bc_Cogent_agemoea.pkl','rb') as f1:
    res_1 = pickle.load(f1)
with open('recent_files/res_bc_Cogent_agemoea2.pkl','rb') as f2:
    res_2 = pickle.load(f2)
with open('recent_files/res_bc_Cogent_nsga2.pkl','rb') as f3:
    res_3 = pickle.load(f3)
with open('recent_files/res_bc_Cogent_nsga3.pkl','rb') as f4:
    res_4 = pickle.load(f4)
with open('recent_files/res_bc_Cogent_rnsga2.pkl','rb') as f5:
    res_5 = pickle.load(f5)
with open('recent_files/res_bc_Cogent_rnsga3.pkl','rb') as f6:
    res_6 = pickle.load(f6)
with open('recent_files/res_bc_Cogent_rvea.pkl','rb') as f7:
    res_7 = pickle.load(f7)
with open('recent_files/res_bc_Cogent_smsemoa.pkl','rb') as f8:
    res_8 = pickle.load(f8)
with open('recent_files/res_bc_Cogent_unsga3.pkl','rb') as f9:
    res_9 = pickle.load(f9)
with open('recent_files/res_bc_Cogent_ctaea.pkl','rb') as f10:
    res_10 = pickle.load(f10)
with open('recent_files/res_bc_Cogent_dnsga2.pkl','rb') as f11:
    res_11 = pickle.load(f11)
with open('recent_files/res_bc_Cogent_kgbdmoea.pkl','rb') as f12:
    res_12 = pickle.load(f12)
with open('recent_files/res_bc_Cogent_moead.pkl','rb') as f13:
    res_13 = pickle.load(f13)

## Check whether all results meet the constranins
- The number of controller >= 2
- The number of Atomix node  >= 3
- The number of controller should be a integer number
- The number of Atomix node should be a integer number

In [3]:
# F = res.F
# print(F[np.argsort(F[:, 2])])
F1=res_1.F
F2=res_2.F
F3=res_3.F
F4=res_4.F
F5=res_5.F
F6=res_6.F
F7=res_7.F
F8=res_8.F
F9=res_9.F
F10 = res_10.F
F11 = res_11.F
F12 = res_12.F
F13 = res_13.F

# Check conditions
c1_agemoea = F1[:, 0] >= 2  # the number of controller >= 2
c2_agemoea = F1[:, 1] >= 3  # the number of Atomix node  >= 3
c3_agemoea = F1[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_agemoea = F1[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_agemoea = F1[:, 2] != np.inf # FST should not be a infinite
c6_agemoea = F1[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_agemoea2 = F2[:, 0] >= 2
c2_agemoea2 = F2[:, 1] >= 3
c3_agemoea2 = F2[:, 0] % 1 == 0 
c4_agemoea2 = F2[:, 1] % 1 == 0
c5_agemoea2 = F2[:, 2] != np.inf
c6_agemoea2 = F2[:, 3] != np.inf 

c1_nsga2 = F3[:, 0] >= 2
c2_nsga2 = F3[:, 1] >= 3
c3_nsga2 = F3[:, 0] % 1 == 0 
c4_nsga2 = F3[:, 1] % 1 == 0
c5_nsga2 = F3[:, 2] != np.inf 
c6_nsga2 = F3[:, 3] != np.inf 

c1_nsga3 = F4[:, 0] >= 2
c2_nsga3 = F4[:, 1] >= 3
c3_nsga3 = F4[:, 0] % 1 == 0 
c4_nsga3 = F4[:, 1] % 1 == 0
c5_nsga3 = F4[:, 2] != np.inf 
c6_nsga3 = F4[:, 3] != np.inf 

c1_rnsga2 = F5[:, 0] >= 2
c2_rnsga2 = F5[:, 1] >= 3
c3_rnsga2 = F5[:, 0] % 1 == 0 
c4_rnsga2 = F5[:, 1] % 1 == 0 
c5_rnsga2 = F5[:, 2] != np.inf 
c6_rnsga2 = F5[:, 3] != np.inf 

c1_rnsga3 = F6[:, 0] >= 2
c2_rnsga3 = F6[:, 1] >= 3
c3_rnsga3 = F6[:, 0] % 1 == 0 
c4_rnsga3 = F6[:, 1] % 1 == 0 
c5_rnsga3 = F6[:, 2] != np.inf 
c6_rnsga3 = F6[:, 3] != np.inf 

c1_rvea = F7[:, 0] >= 2
c2_rvea = F7[:, 1] >= 3
c3_rvea = F7[:, 0] % 1 == 0 
c4_rvea = F7[:, 1] % 1 == 0 
c5_rvea = F7[:, 2] != np.inf
c6_rvea = F7[:, 3] != np.inf 

c1_smsemoa = F8[:, 0] >= 2
c2_smsemoa = F8[:, 1] >= 3
c3_smsemoa = F8[:, 0] % 1 == 0 
c4_smsemoa = F8[:, 1] % 1 == 0 
c5_smsemoa = F8[:, 2] != np.inf
c6_smsemoa = F8[:, 3] != np.inf 

c1_unsga3 = F9[:, 0] >= 2
c2_unsga3 = F9[:, 1] >= 3
c3_unsga3 = F9[:, 0] % 1 == 0 
c4_unsga3 = F9[:, 1] % 1 == 0 
c5_unsga3 = F9[:, 2] != np.inf
c6_unsga3 = F9[:, 3] != np.inf 

c1_ctaea = F10[:, 0] >= 2
c2_ctaea = F10[:, 1] >= 3
c3_ctaea = F10[:, 0] % 1 == 0 
c4_ctaea = F10[:, 1] % 1 == 0 
c5_ctaea = F10[:, 2] != np.inf
c6_ctaea = F10[:, 3] != np.inf 

c1_dnsga2 = F11[:, 0] >= 2
c2_dnsga2 = F11[:, 1] >= 3
c3_dnsga2 = F11[:, 0] % 1 == 0 
c4_dnsga2 = F11[:, 1] % 1 == 0 
c5_dnsga2 = F11[:, 2] != np.inf
c6_dnsga2 = F11[:, 3] != np.inf 

c1_kgbdmoea = F12[:, 0] >= 2
c2_kgbdmoea = F12[:, 1] >= 3
c3_kgbdmoea = F12[:, 0] % 1 == 0 
c4_kgbdmoea = F12[:, 1] % 1 == 0 
c5_kgbdmoea = F12[:, 2] != np.inf
c6_kgbdmoea = F12[:, 3] != np.inf 

c1_moead = F13[:, 0] >= 2
c2_moead = F13[:, 1] >= 3
c3_moead = F13[:, 0] % 1 == 0 
c4_moead = F13[:, 1] % 1 == 0 
c5_moead = F13[:, 2] != np.inf
c6_moead = F13[:, 3] != np.inf 

if np.all(c1_agemoea) and np.all(c2_agemoea) and np.all(c3_agemoea) and np.all(c4_agemoea) and np.all(c5_agemoea) and np.all(c6_agemoea):
    print("agemoea: " + 'True')
else: 
    print("agemoea: " + 'False')

if np.all(c1_agemoea2) and np.all(c2_agemoea2) and np.all(c3_agemoea2) and np.all(c4_agemoea2) and np.all(c5_agemoea2) and np.all(c6_agemoea2):
    print("agemoea2: " + 'True')
else: 
    print("agemoea2: " + 'False')
    
if np.all(c1_nsga2) and np.all(c2_nsga2) and np.all(c3_nsga2) and np.all(c4_nsga2) and np.all(c5_nsga2) and np.all(c6_nsga2):
    print("nsga2: " + 'True')
else: 
    print("nsga2: " + 'False')

if np.all(c1_nsga3) and np.all(c2_nsga3) and np.all(c3_nsga3) and np.all(c4_nsga3) and np.all(c5_nsga3) and np.all(c6_nsga3):
    print("nsga3: " + 'True')
else: 
    print("nsga3: " + 'False')

if np.all(c1_rnsga2) and np.all(c2_rnsga2) and np.all(c3_rnsga2) and np.all(c4_rnsga2) and np.all(c5_rnsga2) and np.all(c6_rnsga2):
    print("rnsga2: " + 'True')
else: 
    print("rnsga2: " + 'False')

if np.all(c1_rnsga3) and np.all(c2_rnsga3) and np.all(c3_rnsga3) and np.all(c4_rnsga3) and np.all(c5_rnsga3) and np.all(c6_rnsga3):
    print("rnsga3: " + 'True')
else: 
    print("rnsga3: " + 'False')

if np.all(c1_rvea) and np.all(c2_rvea) and np.all(c3_rvea) and np.all(c4_rvea) and np.all(c5_rvea) and np.all(c6_rvea):
    print("rvea: " + 'True')
else: 
    print("rvea: " + 'False')
    
if np.all(c1_smsemoa) and np.all(c2_smsemoa) and np.all(c3_smsemoa) and np.all(c4_smsemoa) and np.all(c5_smsemoa) and np.all(c6_smsemoa):
    print("smsemoa: " + 'True')
else: 
    print("smsemoa: " + 'False')
    
if np.all(c1_unsga3) and np.all(c2_unsga3) and np.all(c3_unsga3) and np.all(c4_unsga3) and np.all(c5_unsga3) and np.all(c6_unsga3):
    print("unsga3: " + 'True')
else: 
    print("unsga3: " + 'False')

if np.all(c1_ctaea) and np.all(c2_ctaea) and np.all(c3_ctaea) and np.all(c4_ctaea) and np.all(c5_ctaea) and np.all(c6_ctaea):
    print("ctaea: " + 'True')
else: 
    print("ctaea: " + 'False')
    
if np.all(c1_dnsga2) and np.all(c2_dnsga2) and np.all(c3_dnsga2) and np.all(c4_dnsga2) and np.all(c5_dnsga2) and np.all(c6_dnsga2):
    print("dnsga2: " + 'True')
else: 
    print("dnsga2: " + 'False')

if np.all(c1_kgbdmoea) and np.all(c2_kgbdmoea) and np.all(c3_kgbdmoea) and np.all(c4_kgbdmoea) and np.all(c5_kgbdmoea) and np.all(c6_kgbdmoea):
    print("kgbdmoea: " + 'True')
else: 
    print("kgbdmoea: " + 'False')

if np.all(c1_moead) and np.all(c2_moead) and np.all(c3_moead) and np.all(c4_moead) and np.all(c5_moead) and np.all(c6_moead):
    print("moead: " + 'True')
else: 
    print("moead: " + 'False')

agemoea: True
agemoea2: True
nsga2: True
nsga3: True
rnsga2: True
rnsga3: True
rvea: False
smsemoa: True
unsga3: True
ctaea: False
dnsga2: True
kgbdmoea: True
moead: False


## New Implementation

In [5]:
with open('res_bc_Cogent_agemoea.pkl','rb') as f1:
    res_1 = pickle.load(f1)
with open('res_bc_Cogent_agemoea2.pkl','rb') as f2:
    res_2 = pickle.load(f2)
with open('res_bc_Cogent_nsga2.pkl','rb') as f3:
    res_3 = pickle.load(f3)
with open('res_bc_Cogent_nsga3.pkl','rb') as f4:
    res_4 = pickle.load(f4)
with open('res_bc_Cogent_rnsga2.pkl','rb') as f5:
    res_5 = pickle.load(f5)
with open('res_bc_Cogent_rnsga3.pkl','rb') as f6:
    res_6 = pickle.load(f6)
with open('res_bc_Cogent_rvea.pkl','rb') as f7:
    res_7 = pickle.load(f7)
with open('res_bc_Cogent_smsemoa.pkl','rb') as f8:
    res_8 = pickle.load(f8)
with open('res_bc_Cogent_unsga3.pkl','rb') as f9:
    res_9 = pickle.load(f9)
with open('res_bc_Cogent_ctaea.pkl','rb') as f10:
    res_10 = pickle.load(f10)
with open('res_bc_Cogent_dnsga2.pkl','rb') as f11:
    res_11 = pickle.load(f11)
with open('res_bc_Cogent_kgbdmoea.pkl','rb') as f12:
    res_12 = pickle.load(f12)
with open('res_bc_Cogent_moead.pkl','rb') as f13:
    res_13 = pickle.load(f13)

In [6]:
# F = res.F
# print(F[np.argsort(F[:, 2])])
F1=res_1.F
F2=res_2.F
F3=res_3.F
F4=res_4.F
F5=res_5.F
F6=res_6.F
F7=res_7.F
F8=res_8.F
F9=res_9.F
F10 = res_10.F
F11 = res_11.F
F12 = res_12.F
F13 = res_13.F

# Check conditions
c1_agemoea = F1[:, 0] >= 2  # the number of controller >= 2
c2_agemoea = F1[:, 1] >= 3  # the number of Atomix node  >= 3
c3_agemoea = F1[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_agemoea = F1[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_agemoea = F1[:, 2] != np.inf # FST should not be a infinite
c6_agemoea = F1[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_agemoea2 = F2[:, 0] >= 2
c2_agemoea2 = F2[:, 1] >= 3
c3_agemoea2 = F2[:, 0] % 1 == 0 
c4_agemoea2 = F2[:, 1] % 1 == 0
c5_agemoea2 = F2[:, 2] != np.inf
c6_agemoea2 = F2[:, 3] != np.inf 

c1_nsga2 = F3[:, 0] >= 2
c2_nsga2 = F3[:, 1] >= 3
c3_nsga2 = F3[:, 0] % 1 == 0 
c4_nsga2 = F3[:, 1] % 1 == 0
c5_nsga2 = F3[:, 2] != np.inf 
c6_nsga2 = F3[:, 3] != np.inf 

c1_nsga3 = F4[:, 0] >= 2
c2_nsga3 = F4[:, 1] >= 3
c3_nsga3 = F4[:, 0] % 1 == 0 
c4_nsga3 = F4[:, 1] % 1 == 0
c5_nsga3 = F4[:, 2] != np.inf 
c6_nsga3 = F4[:, 3] != np.inf 

c1_rnsga2 = F5[:, 0] >= 2
c2_rnsga2 = F5[:, 1] >= 3
c3_rnsga2 = F5[:, 0] % 1 == 0 
c4_rnsga2 = F5[:, 1] % 1 == 0 
c5_rnsga2 = F5[:, 2] != np.inf 
c6_rnsga2 = F5[:, 3] != np.inf 

c1_rnsga3 = F6[:, 0] >= 2
c2_rnsga3 = F6[:, 1] >= 3
c3_rnsga3 = F6[:, 0] % 1 == 0 
c4_rnsga3 = F6[:, 1] % 1 == 0 
c5_rnsga3 = F6[:, 2] != np.inf 
c6_rnsga3 = F6[:, 3] != np.inf 

c1_rvea = F7[:, 0] >= 2
c2_rvea = F7[:, 1] >= 3
c3_rvea = F7[:, 0] % 1 == 0 
c4_rvea = F7[:, 1] % 1 == 0 
c5_rvea = F7[:, 2] != np.inf
c6_rvea = F7[:, 3] != np.inf 

c1_smsemoa = F8[:, 0] >= 2
c2_smsemoa = F8[:, 1] >= 3
c3_smsemoa = F8[:, 0] % 1 == 0 
c4_smsemoa = F8[:, 1] % 1 == 0 
c5_smsemoa = F8[:, 2] != np.inf
c6_smsemoa = F8[:, 3] != np.inf 

c1_unsga3 = F9[:, 0] >= 2
c2_unsga3 = F9[:, 1] >= 3
c3_unsga3 = F9[:, 0] % 1 == 0 
c4_unsga3 = F9[:, 1] % 1 == 0 
c5_unsga3 = F9[:, 2] != np.inf
c6_unsga3 = F9[:, 3] != np.inf 

c1_ctaea = F10[:, 0] >= 2
c2_ctaea = F10[:, 1] >= 3
c3_ctaea = F10[:, 0] % 1 == 0 
c4_ctaea = F10[:, 1] % 1 == 0 
c5_ctaea = F10[:, 2] != np.inf
c6_ctaea = F10[:, 3] != np.inf 

c1_dnsga2 = F11[:, 0] >= 2
c2_dnsga2 = F11[:, 1] >= 3
c3_dnsga2 = F11[:, 0] % 1 == 0 
c4_dnsga2 = F11[:, 1] % 1 == 0 
c5_dnsga2 = F11[:, 2] != np.inf
c6_dnsga2 = F11[:, 3] != np.inf 

c1_kgbdmoea = F12[:, 0] >= 2
c2_kgbdmoea = F12[:, 1] >= 3
c3_kgbdmoea = F12[:, 0] % 1 == 0 
c4_kgbdmoea = F12[:, 1] % 1 == 0 
c5_kgbdmoea = F12[:, 2] != np.inf
c6_kgbdmoea = F12[:, 3] != np.inf 

c1_moead = F13[:, 0] >= 2
c2_moead = F13[:, 1] >= 3
c3_moead = F13[:, 0] % 1 == 0 
c4_moead = F13[:, 1] % 1 == 0 
c5_moead = F13[:, 2] != np.inf
c6_moead = F13[:, 3] != np.inf 

if np.all(c1_agemoea) and np.all(c2_agemoea) and np.all(c3_agemoea) and np.all(c4_agemoea) and np.all(c5_agemoea) and np.all(c6_agemoea):
    print("agemoea: " + 'True')
else: 
    print("agemoea: " + 'False')

if np.all(c1_agemoea2) and np.all(c2_agemoea2) and np.all(c3_agemoea2) and np.all(c4_agemoea2) and np.all(c5_agemoea2) and np.all(c6_agemoea2):
    print("agemoea2: " + 'True')
else: 
    print("agemoea2: " + 'False')
    
if np.all(c1_nsga2) and np.all(c2_nsga2) and np.all(c3_nsga2) and np.all(c4_nsga2) and np.all(c5_nsga2) and np.all(c6_nsga2):
    print("nsga2: " + 'True')
else: 
    print("nsga2: " + 'False')

if np.all(c1_nsga3) and np.all(c2_nsga3) and np.all(c3_nsga3) and np.all(c4_nsga3) and np.all(c5_nsga3) and np.all(c6_nsga3):
    print("nsga3: " + 'True')
else: 
    print("nsga3: " + 'False')

if np.all(c1_rnsga2) and np.all(c2_rnsga2) and np.all(c3_rnsga2) and np.all(c4_rnsga2) and np.all(c5_rnsga2) and np.all(c6_rnsga2):
    print("rnsga2: " + 'True')
else: 
    print("rnsga2: " + 'False')

if np.all(c1_rnsga3) and np.all(c2_rnsga3) and np.all(c3_rnsga3) and np.all(c4_rnsga3) and np.all(c5_rnsga3) and np.all(c6_rnsga3):
    print("rnsga3: " + 'True')
else: 
    print("rnsga3: " + 'False')

if np.all(c1_rvea) and np.all(c2_rvea) and np.all(c3_rvea) and np.all(c4_rvea) and np.all(c5_rvea) and np.all(c6_rvea):
    print("rvea: " + 'True')
else: 
    print("rvea: " + 'False')
    
if np.all(c1_smsemoa) and np.all(c2_smsemoa) and np.all(c3_smsemoa) and np.all(c4_smsemoa) and np.all(c5_smsemoa) and np.all(c6_smsemoa):
    print("smsemoa: " + 'True')
else: 
    print("smsemoa: " + 'False')
    
if np.all(c1_unsga3) and np.all(c2_unsga3) and np.all(c3_unsga3) and np.all(c4_unsga3) and np.all(c5_unsga3) and np.all(c6_unsga3):
    print("unsga3: " + 'True')
else: 
    print("unsga3: " + 'False')

if np.all(c1_ctaea) and np.all(c2_ctaea) and np.all(c3_ctaea) and np.all(c4_ctaea) and np.all(c5_ctaea) and np.all(c6_ctaea):
    print("ctaea: " + 'True')
else: 
    print("ctaea: " + 'False')
    
if np.all(c1_dnsga2) and np.all(c2_dnsga2) and np.all(c3_dnsga2) and np.all(c4_dnsga2) and np.all(c5_dnsga2) and np.all(c6_dnsga2):
    print("dnsga2: " + 'True')
else: 
    print("dnsga2: " + 'False')

if np.all(c1_kgbdmoea) and np.all(c2_kgbdmoea) and np.all(c3_kgbdmoea) and np.all(c4_kgbdmoea) and np.all(c5_kgbdmoea) and np.all(c6_kgbdmoea):
    print("kgbdmoea: " + 'True')
else: 
    print("kgbdmoea: " + 'False')

if np.all(c1_moead) and np.all(c2_moead) and np.all(c3_moead) and np.all(c4_moead) and np.all(c5_moead) and np.all(c6_moead):
    print("moead: " + 'True')
else: 
    print("moead: " + 'False')

agemoea: True
agemoea2: True
nsga2: True
nsga3: True
rnsga2: True
rnsga3: True
rvea: False
smsemoa: True
unsga3: True
ctaea: True
dnsga2: False
kgbdmoea: False
moead: False


: 